In [1]:
from functions_py import convert_to_polar, polar_conversion, func_def, fourier_coef_calc, get_label, img_plot, polar_conversion_fast, get_label_fast
from functions_py import pipeline_train, batch_get_labels, optimized_test_class_alg
from functions_py import optimized_func_def, optimized_fourier_coef_calc
from functions_py import qwen_pipeline_train, qwen_optimized_test_class_alg, qwen_optimized_func_def, qwen_optimized_fourier_coef_calc, qwen_polar_conversion_fast
from functions_py import run_hyperparameter_optimization
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from numba import njit
from scipy.spatial.distance import cdist


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import os

In [10]:
def transform_pixels_to_sums_vectorized(df):
    # Создаем копию исходного DataFrame
    result_df = df[['label']].copy()
    
    # Получаем все пиксельные колонки
    pixel_cols = [f'pixel{i}' for i in range(784)]
    pixels = df[pixel_cols].values
    
    # Преобразуем в 3D массив (n_images, 28, 28)
    images = pixels.reshape(-1, 28, 28)
    
    # Суммы по строкам (axis=2 - сумма по столбцам в каждом ряду)
    row_sums = images.sum(axis=2)
    
    # Суммы по столбцам (axis=1 - сумма по рядам в каждом столбце)
    col_sums = images.sum(axis=1)
    
    # Добавляем суммы в DataFrame
    for i in range(28):
        result_df[f'sum_row{i}'] = row_sums[:, i] / 28
    for i in range(28):
        result_df[f'sum_column{i}'] = col_sums[:, i] / 28
    
    return result_df

from skimage.feature import hog

def extract_hog_features(df, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1)):
    result_df = df[['label']].copy()
    pixel_cols = [f'pixel{i}' for i in range(784)]
    images = df[pixel_cols].values.reshape(-1, 28, 28)
    
    hog_features = []
    for image in images:
        features = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block, feature_vector=True)
        hog_features.append(features)
    
    hog_array = np.array(hog_features)
    for i in range(hog_array.shape[1]):
        result_df[f'hog_{i}'] = hog_array[:, i]
    
    return result_df

In [11]:
input_data = pd.read_csv(base + 'train.csv')
extract_hog_features(input_data)

,label,hog_0,hog_1,hog_2,hog_3,hog_4,hog_5,hog_6,hog_7,hog_8,...,hog_62,hog_63,hog_64,hog_65,hog_66,hog_67,hog_68,hog_69,hog_70,hog_71
0,1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.272452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,0.698176,0.698176,0.158430,0.0,0.000000,0.000000,0.000000,0.000000,0.194846,...,0.528689,0.049075,0.521107,0.521107,0.521107,0.418958,0.099082,0.000000,0.000000,0.000000
2,1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.455339,...,0.374449,0.087337,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,4,0.447214,0.000000,0.000000,0.0,0.447214,0.447214,0.447214,0.447214,0.117965,...,0.377964,0.377964,0.410242,0.388405,0.352563,0.257576,0.270947,0.283173,0.410242,0.410242
4,0,0.702586,0.702586,0.112901,0.0,0.000000,0.000000,0.000000,0.000000,0.223686,...,0.416403,0.133872,0.231908,0.556170,0.556170,0.556170,0.135065,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.135668,...,0.000000,0.000000,0.335901,0.623326,0.623326,0.331646,0.010551,0.000000,0.000000,0.000000
41996,1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.577040,...,0.000000,0.345001,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
41997,7,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.577350,0.577350,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000
41998,6,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.577315,...,0.389142,0.228789,0.474904,0.474904,0.474904,0.474904,0.190928,0.000000,0.236093,0.075298


In [ ]:
base = ''
input_data = pd.read_csv(base + 'train.csv')
conversion_state = True
write_state = False
state = 148
target = 'label'

prefix = 'COEF_TEST_1.05_column_row_'
D_left = 10
D_right = 15000
M_left = 2
M_right = 51
T_left = 0.1
T_right = 10.0

max_iter = 5000

best_param, best_val = run_hyperparameter_optimization(extract_hog_features, input_data, target, 
                                                       conversion_state, write_state, state, base, prefix, D_left, D_right, 
                                                       M_left, M_right, T_left, T_right, max_iter)
print('param', best_param)
print('acc = ', best_val)

[I 2025-05-28 18:53:02,594] A new study created in memory with name: no-name-196b6ff1-255b-43e6-be80-9f1984f1b6f0


Тестируем параметры: D=11536, M=17, t=5.700618640095918


[I 2025-05-28 18:55:15,701] Trial 0 finished with value: 0.12676767676767678 and parameters: {'D': 11536, 'M': 17, 't': 5.700618640095918}. Best is trial 0 with value: 0.12676767676767678.


Тестируем параметры: D=11533, M=23, t=1.2858581560826967


[I 2025-05-28 18:57:46,636] Trial 1 finished with value: 0.6006493506493507 and parameters: {'D': 11533, 'M': 23, 't': 1.2858581560826967}. Best is trial 1 with value: 0.6006493506493507.


...

Тестируем параметры: D=4694, M=24, t=1.2425717095222273


[I 2025-05-29 07:55:08,525] Trial 322 finished with value: 0.6085137085137086 and parameters: {'D': 4694, 'M': 24, 't': 1.2425717095222273}. Best is trial 55 with value: 0.6225108225108225.


Тестируем параметры: D=3485, M=24, t=1.0423567582586284


[I 2025-05-29 07:56:18,758] Trial 323 finished with value: 0.3686868686868687 and parameters: {'D': 3485, 'M': 24, 't': 1.0423567582586284}. Best is trial 55 with value: 0.6225108225108225.


Тестируем параметры: D=4615, M=24, t=1.241688631067533


In [ ]:
base = ''
input_data = pd.read_csv(base + 'train.csv')
conversion_state = True
write_state = False
state = 148
target = 'label'

prefix = 'COEF_TEST_1.05_column_row_'
D_left = 2800
D_right = 2821
M_left = 2
M_right = 11
t_t = [1 + i/100 for i in range(11)]
t_coef_left = t_t[0]
t_coef_right = t_t[-1]

t_str = 'T_from_' + str(t_coef_left) + '_to_' + str(t_coef_right)
res_name = prefix + t_str + 'D_from_' + str(D_left) + '_to_' + str(D_right - 1) + '_M_from_' + str(M_left) + '_to_' + str(M_right - 1)
res_file_path = os.path.join('logs', f'results_{res_name}.txt')

best_acc = 0

for t_coef in t_t:
    for D_val in range (D_left, D_right):
        for M_val in range(M_left, M_right):
            print('---------------D = ', D_val , 'M = ', M_val, 't =', t_coef, '----------------')
            data_with_labels = transform_pixels_to_sums_vectorized(input_data)
            
            D = D_val
            M = M_val
            file_temp_name = prefix + 'D'+ str(D_val) + 'M' + str(M_val)
            
            data_train, data_test, data_train_means, fourier_coef = qwen_pipeline_train(data_with_labels, target, state, conversion_state, write_state, D, M)
            accuracy = qwen_optimized_test_class_alg(prefix, file_temp_name, target,data_test, data_train, data_train_means, fourier_coef, t_coef, rewrite = True)

            print('acc = ', accuracy)
            if accuracy > best_acc:
                best_acc = accuracy
            
            print('---------------D = ', D_val , 'M = ', M_val, 't =', t_coef, '----------------', file=open(res_file_path, 'a'))
            print('accuracy = ', accuracy , file=open(res_file_path, 'a'))
    print('BEST ACC = ', best_acc)

---------------D =  2800 M =  2 t = 1.0 ----------------
acc =  0.42193362193362194
---------------D =  2800 M =  3 t = 1.0 ----------------
acc =  0.4997113997113997
---------------D =  2800 M =  4 t = 1.0 ----------------
acc =  0.49213564213564215
---------------D =  2800 M =  5 t = 1.0 ----------------
acc =  0.5562770562770563
---------------D =  2800 M =  6 t = 1.0 ----------------
acc =  0.5808802308802309
---------------D =  2800 M =  7 t = 1.0 ----------------
acc =  0.5391053391053391
---------------D =  2800 M =  8 t = 1.0 ----------------
acc =  0.5681096681096681
---------------D =  2800 M =  9 t = 1.0 ----------------
acc =  0.5823232323232324
---------------D =  2800 M =  10 t = 1.0 ----------------
acc =  0.554040404040404
---------------D =  2801 M =  2 t = 1.0 ----------------
acc =  0.422005772005772
---------------D =  2801 M =  3 t = 1.0 ----------------
acc =  0.4980519480519481
---------------D =  2801 M =  4 t = 1.0 ----------------
acc =  0.48968253968253966
--

In [12]:
def test_fixed_T(res_file_path, prefix, t_coef, D_left, D_right, M_left, M_right, input_data, target, state, conversion_state, write_state):
    best_acc = 0
    for D_val in range (D_left, D_right):
        for M_val in range(M_left, M_right):
            print('---------------D = ', D_val , 'M = ', M_val, 't =', t_coef, '----------------')
            data_with_labels = transform_pixels_to_sums_vectorized(input_data)
            
            D = D_val
            M = M_val
            file_temp_name = prefix + 'D'+ str(D_val) + 'M' + str(M_val)
            
            data_train, data_test, data_train_means, fourier_coef = pipeline_train(data_with_labels, target, state, conversion_state, write_state, D, M)
            accuracy, precision = optimized_test_class_alg(prefix, file_temp_name, target,data_test, data_train, data_train_means, fourier_coef, t_coef, rewrite = True)
            
            print('accuracy = ', accuracy)
            
            if accuracy > best_acc:
                best_acc = accuracy
            
            print('---------------D = ', D_val , 'M = ', M_val, 't =', t_coef, '----------------', file=open(res_file_path, 'a'))
            print('accuracy = ', accuracy , file=open(res_file_path, 'a'))
    print('BEST ACC = ', best_acc)
    return best_acc

In [5]:
base = ''
input_data = pd.read_csv(base + 'train.csv')
conversion_state = True
write_state = False
state = 148
target = 'label'

prefix = 'Dihot_T_test_1_to_1.01_column_row_'
D_left = 2
D_right = 151
M_left = 2
M_right = 11
t_left = 1
t_right = 1.1

best_res = 

t_str = 'T_from_' + str(t_coef_left) + '_to_' + str(t_coef_right)
res_name = prefix + t_str + 'D_from_' + str(D_left) + '_to_' + str(D_right - 1) + '_M_from_' + str(M_left) + '_to_' + str(M_right - 1)
res_file_path = os.path.join('logs', f'results_{res_name}.txt')
# for t_coef in t_t:
#     print('T = ', t_coef):
#     t_res = test_fixed_T(res_file_path, t_coef, D_left, D_right, M_left, M_right, input_data)
#     print('best acc = ', t_res)

[1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1]


In [ ]:
from scipy.optimize import golden

def accuracy_func(t_coef):
    return -test_fixed_T(res_file_path, prefix, t_coef, D_left, D_right, M_left, M_right, input_data, target, state, conversion_state, write_state)

base = ''
input_data = pd.read_csv(base + 'train.csv')
conversion_state = True
write_state = False
state = 148
target = 'label'

prefix = 'Golden_T_test_1_to_1.01_column_row_'
D_left = 2
D_right = 101
M_left = 2
M_right = 11
t_left = 1
t_right = 1.1

t_str = 'T_from_' + str(t_coef_left) + '_to_' + str(t_coef_right)
res_name = prefix + t_str + 'D_from_' + str(D_left) + '_to_' + str(D_right - 1) + '_M_from_' + str(M_left) + '_to_' + str(M_right - 1)
res_file_path = os.path.join('logs', f'results_{res_name}.txt')


# Используем scipy.optimize.golden для поиска минимума (поэтому передаём -accuracy)
best_t = golden(accuracy_func, brack=(t_left, t_right), tol=0.001)
best_accuracy = -accuracy_func(best_t)

---------------D =  2 M =  2 t = 1.0 ----------------
——— Converting to polar ———
——— Calculating piece-wise function ———
——— Calculating fourier coefficients ———
——— Done ! ———
——— Testing is over ———
accuracy =  0.15353535353535352
---------------D =  2 M =  3 t = 1.0 ----------------
——— Converting to polar ———
——— Calculating piece-wise function ———
——— Calculating fourier coefficients ———
——— Done ! ———
——— Testing is over ———
accuracy =  0.15353535353535352
---------------D =  2 M =  4 t = 1.0 ----------------
——— Converting to polar ———
——— Calculating piece-wise function ———
——— Calculating fourier coefficients ———
——— Done ! ———
——— Testing is over ———
accuracy =  0.25072150072150073
---------------D =  2 M =  5 t = 1.0 ----------------
——— Converting to polar ———
——— Calculating piece-wise function ———
——— Calculating fourier coefficients ———
——— Done ! ———
——— Testing is over ———
accuracy =  0.25072150072150073
---------------D =  2 M =  6 t = 1.0 ----------------
——— Conver